In [1]:
pip install tflearn==0.3.2

In [2]:
pip install numpy==1.16.5

In [3]:
pip install tensorflow==1.13.2

In [4]:
pip install nltk==3.4.5

In [5]:
import nltk
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
print(np.__version__)
print(nltk.__version__)
print(tf.__version__)

1.16.5
3.4.5
1.13.2


In [10]:
ls 

sample_data/


In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
ls 

gdrive/  sample_data/


In [10]:
cd gdrive

/content/gdrive


In [11]:
cd MyDrive/

/content/gdrive/MyDrive


In [15]:
ls 

'20210409_185439_(1)[1].jpg'
'Adding Functionalities to DeepLense for Decoding Dark Matter.gdoc'
'AI Game '/
'AI Game pictures'/
 appdata/
'appdata (1)'/
'Certificates '/
'Chat-bot project '/
 Classroom/
'CO-Assignment 1.docx'
'Colab Notebooks'/
'computer organisation assignment 1 .pdf'
'DSC ML interview question.gsheet'
'ESCS 391 Question.gdoc'
'ESCS 391 Viva Schedule.gsheet'
'Getting started.pdf'
 Letter.gdoc
'logistic regression ( application video ).wmv'
 LR.mp4
'My pictures'/
 my_resume.pdf
'PROGRAM OUTPUT .gslides'
'Resume (2) (1).pdf'
'Resume (2).pdf'
'Resume folder '/
 Resume.pdf
'Sayantan Sadhu 10200219005 economics .pdf'
'Sayantan_Sadhu_ 10200219005.gdoc'
 Sayantan_Sadhu_10200219005.pdf
'Sayantan_Sadhu_10200219005_python_assignment .docx'
 SayantanSadhu_BaggingTechnique.gdoc
 SayantanSadhu_BaggingTechnique.pdf
'Session presentation .gslides'
'Syed Taslim Kawser,10200219046,IT,DS & Algo PCA2.pptx'
'Untitled document (1).gdoc'
'Untitled document (2).gdoc'
'Untitled document.gdo

In [12]:
cd 'Chat-bot_project '

/content/gdrive/MyDrive/Chat-bot_project 


In [21]:
ls

Data.json  Untitled0.ipynb


In [13]:
with open("Data.json") as file:
	data = json.load(file)

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
words = []
labels = []
docs_x = []
docs_y = []

#Looping through our data
for intent in data['intents']:
	for pattern in intent['patterns']:
		pattern = pattern.lower()
    		#Creating a list of words
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent['tag'])

	if intent['tag'] not in labels:
	  labels.append(intent['tag'])

In [20]:
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1
	training.append(bag)
	output.append(output_row)
#Converting training data into NumPy arrays
training = np.array(training)
output = np.array(output)

#Saving data to disk
with open("data.pickle","wb") as f:
	pickle.dump((words, labels, training, output),f)

In [21]:
tf.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [22]:
model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)


Training Step: 1399  | total loss: 0.16795 | time: 0.059s
| Adam | epoch: 200 | loss: 0.16795 - acc: 1.0000 -- iter: 48/55
Training Step: 1400  | total loss: 0.16751 | time: 0.072s
| Adam | epoch: 200 | loss: 0.16751 - acc: 1.0000 -- iter: 55/55
--


In [23]:
model.save("model.tflearn")

INFO:tensorflow:/content/gdrive/My Drive/Chat-bot_project /model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
